# Sesiones prácticas

## 0

Instalación de Python + ecosistema científico + opencv + opengl

- aula virtual -> página web -> install
- git o unzip master
- anaconda completo o miniconda
- linux, windows, mac
- probar scripts con webcam y verificar opengl, dlib etc.
- manejo básico de jupyter
- repaso Python
- Ejercicio: recortar y unir imágenes para conseguir [algo como esto](../images/demos/ej-c0.png).

Opcional:

- compilación opencv
- docker

## 1

Ejercicio de comprobación de FOV/tamaños/distancias.

Dispositivos de captura

- umucv (install con --upgrade) (update_umucv.sh)
- webcam.py con opencv crudo
- stream.py, opciones de autostream, efecto de teclas, --help, --dev=help

    - webcams
    - videos
    - carpeta de imágenes
    - teléfono
    - youtube
    - urls de tv
        
- ejemplo de recorte invertido

## 2

Más utilidades

- spyder

- PYTHONPATH

- control de webcam v4l2-ctl, vlc, gucview

- wzoom.py (para las ventanas de Windows que no tienen zoom)

- help_window.py

- save_video.py

- ampliar mouse.py:

    - círculos en las posiciones marcadas (cv.circle)
    
    - coordenadas textuales (cv.putText  (ej. en hello.py) o umucv.util.putText)

    - marcar solo los dos últimos (pista: collections.deque)

    - reproducir code/medidor.py indicando la distancia en pixels

    - Dado el FOV: indicar el ángulo de las direcciones marcadas.

## 3

deque.py

roi.py:

- añadir la media del nivel de gris del recorte

- guardar el recorte y mostrar cv.absdiff respecto al frame actual, mostrando su media o su máximo.

(Sirve de punto de partida para el ejercicio ACTIVIDAD)

## 4

- aclaraciones ejercicio COLOR

- demo spectral

- trackbar.py

- demo filtros

Ejercicio: 

- implementación de filtro gaussiano con tracker sigma en toda la imagen, monocromo ([ejemplo](../images/demos/ej-c4-0.png)).
- add box y median
- medir y mostrar tiempos de cómputo en diferentes casos

(Sirve de punto de partida para el ejercicio opcional FILTROS)

## 5

HOG

- (captura asíncrona)

- (teoría de HOG, implementación sencilla)

- hog0.py en detalle

- pedestrian.py, detección multiescala

- DLIB facelandmarks.py: HOG face detector con landmarks

Ejercicio: blink detection, inpainting eyes, etc.

## 6

Detección de corners y Flujo óptico de Lucas-Kanade

- LK/*.py


Vamos a construir un "tracker" de puntos de interés basado en el método de Lucas-Kanade.

El primer paso es construir un detector de corners partiendo de cero, calculando la imagen de respuesta correspondiente al menor valor propio de la matriz de covarianza de la distribución local del gradiente en cada pixel (`corners0.py`). En realidad esta operación está directamente disponible en opencv mediante cv.goodFeaturesToTrack (`corners1.py`, `corners2.py ).

El siguiente ejemplo muestra cómo encontrar directamente con `cv.calcOpticalFlowPyrLK` la posición de los puntos detectados en el fotograma siguiente, sin necesidad de recalcular puntos nuevos y asociarlos con los anteriores (`lk_track0.py`).

A continuación ampliamos el código para generar puntos nuevos periódicamente y crear una lista de trayectorias "tracks" que se mantiene actualizada en cada fotograma (`lk_track1.py`).

Finalmente, ampliamos el código anterior para que solo se generen puntos nuevos en zonas de la imagen donde no los haya, y mejoramos la detección de las posiciones siguientes con un criterio de calidad muy robusto que exige que la predicción hacia el pasado de los puntos nuevos coincida con el punto inicial. Si no hay una asociación mutua el punto y su trayectoria se descartan (`lk_tracks.py`).

Ejercicios:

- Analizar los tracks para determinar en qué dirección se mueve la cámara (UP,DOWN,LEFT,RIGHT, [FORWARD, BACKWARD])

- Estudiar la posibilidad de hacer tracking de un ROI.

## 7

Experimentamos con el detector de puntos de interés SIFT.

Nuestro objetivo es obtener un conjunto de "keypoints", cada uno con su descriptor (vector de características que describe el entorno del punto), que permita encontrarlo en imágenes futuras. Esto tiene una aplicación inmediata para reconocer objetos y más adelante en geometría visual.

Empezamos con el ejemplo de código code/SIFT/sift0.py, que simplemente calcula y muestra los puntos de interés. Es interesante observar el efecto de los parámetros del método y el tiempo de cómputo en función del tamaño de la imagen (que puedes cambiar con --size o --resize).

El siguiente ejemplo code/SIFT/sift1.py muestra un primer ataque para establecer correspondencias. Los resultados son bastante pobres porque se aceptan todas las posibles coincidencias.

Finalmente, en code/SIFT/sift.py aplicamos un criterio de selección para eliminar muchas correspondencias erróneas (aunque no todas). Esto es en principio suficiente para el reconocimiento de objetos. (Más adelante veremos una forma mucho mejor de eliminar correspondencias erróneas, necesaria para aplicaciones de geometría.)

El ejercicio obligatorio **SIFT** es una ampliación sencilla de este código. Se trata de almacenar un conjunto de modelos (¡con textura! para que tengan suficientes keypoints) como portadas de libros, discos, videojuegos, etc. y reconocerlos en base a la proporción de coincidencias detectadas.

En la segunda parte de la clase experimentamos con un servidor mjpg y creamos bots de telegram (explicados al final de este documento) para comunicarnos fácilmente con las aplicaciones de visión artificial desde el móvil.

## 8

Reconocimiento de formas mediante descriptores frecuenciales.

Nuestro objetivo es hacer un programa que reconozca la forma de trébol, como se muestra [en este pantallazo](../images/demos/shapedetect.png). Si no tenéis a mano un juego de cartas podéis usar --dev=dir:../images/card*.png para hacer las pruebas, aunque lo ideal es hacerlo funcionar con una cámara en vivo.

Trabajaremos con los ejemplos de código de la carpeta `code/shapes` y, como es habitual, iremos añadiendo poco a poco funcionalidad. En cada nuevo paso los comentarios explican los cambios respecto al paso anterior.

Empezamos con el ejemplo shapes/trebol1.py, que simplemente prepara un bucle de captura básico, binariza la imagen y muestra los contornos encontrados. Se muestran varias formas de realizar la binarización y se puede experimentar con ellas, pero en principio el método automático propuesto suele funcionar bien en muchos casos.

El segundo paso en shapes/trebol2.py junta la visualización en una ventana y selecciona los contornos oscuros de tamaño razonable. Esto no es imprescincible para nuestra aplicación, pero es conveniente familiarizarse con el concepto de orientación de un contorno.

En shapes/trebol3.py leemos un modelo de la silueta trébol de una imagen que tenemos en el repositorio y la mostramos en una ventana.

En shapes/trebol3b.py hacemos una utilidad para ver gráficamente las componentes frecuenciales como elipses que componen la figura. Podemos ver las componentes en su tamaño natural, incluyendo la frecuencia principal, [como aquí](../images/demos/full-components.png), o quitando la frecuencia principal y ampliando el tamaño de las siguientes, que son la base del descriptor de forma, [como se ve aquí](../images/demos/shape-components.png). Observa que las configuraciones de elipses son parecidas cuando corresponden a la misma silueta.

En shapes/trebol4.py definimos la función que calcula el descriptor invariante. Se basa esencialmente en calcular los tamaños relativos de estas elipses. En el código se explica cómo se consigue la invarianza a las transformaciones deseadas: posición, tamaño, giros, punto de partida del contorno y ruido de medida.

Finalmente, en shapes/trebol5.py calculamos el descriptor del modelo y en el bucle de captura calculamos los descriptores de los contornos oscuros detectados para marcar las siluetas que tienen un descriptor muy parecido al del trébol.

## 8b

En esta subsesión vamos a hacer varias actividades. Necesitamos algunos paquetes. En Linux son:

    sudo apt install tesseract-ocr tesseract-ocr-spa libtesseract-dev
    pip install tesserocr

    sudo apt install libzbar-dev
    pip install pyzbar

[zbar Windows instaler](http://zbar.sourceforge.net) -> download

Usuarios de Mac y Windows: investigad la forma de instalar tesseract.


1) En primer lugar nos fijamos en el script [code/ocr.py](../code/ocr.py), cuya misión es poner en marcha el OCR con la cámara en vivo. Usamos el paquete de python `tesserocr`. Vamos a verificar el funcionamiento con una imagen estática, pero lo ideal es probarlo con la cámara en vivo.

    ./ocr.py --dev=dir:../images/texto/bo0.png 

Está pensado para marcar una sola línea de texto, [como se muestra aquí](../images/demos/ocr.png). Este pantallazo se ha hecho con la imagen bo1.png disponible en la misma carpeta, que está desenfocada, pero aún así el OCR funciona bien.

(En windows parece que hay que usar pytesseract en lugar de tesserocr, lo que requiere adaptar del script.)

Para mostrar la complejidad de un ocr mostramos el resultado del script `crosscorr.py` sobre images/texto.png, para observar que la comparación pixel a pixel no es suficiente para obtener resultados satisfactorios. En esa misma imagen la binarización y extracción de componentes conexas no consigue separar letras individuales.

Finalmente demostramos mediante `spectral.py` que la transormada de Fourier 2D permite detectar el ángulo y la separación entre renglones.

2) El segundo ejemplo es `code/zbardemo.png` que muestra el uso del paquete pyzbar para leer códigos de barras ([ejemplo](../images/demos/barcode.png)) y códigos QR ([ejemplo](../images/demos/qr.png)) con la cámara. En los códigos de barras se detectan puntos de referencia, y en los QR se detectan las 4 esquinas del cuadrado, que pueden ser útiles como referencia en algunas aplicaciones de geometría.

4) demo de `grabcut.py` para segmentar interactivamente una imagen. Lo probamos con images/puzzle3.png.

5) Ponemos en marcha el detector de caras de opencv con la webcam en vivo y comparamos con el detector de DLIB.

## 9

Hoy vamos a rectificar el plano de la mesa apoyándonos en marcadores artificiales.

En primer lugar trabajaremos con marcadores poligonales. Nuestro objetivo es detectar un marcador como el que aparece en el vídeo `images/rot4.mjpg`. Nos vamos a la carpeta `code/polygon`.

El primer paso (`polygon0.py`) es detectar figuras poligonales con el número de lados correcto a partir de los contornos detectados.

A continuación (`polygon1.py`) nos quedamos con los polígonos que realmente pueden corresponder al marcador. Esto se hace viendo si existe una homografía que relaciona con precisión suficiente el marcador real y su posible imagen.

Finalmente (`polygon2.py`) obtiene el plano rectificado

También se puede añadir información "virtual" a la imagen original, como por ejemplo los ejes de coordenadas definidos por el marcador (`polygon3.py`).


Como segunda actividad, en la carpeta `code/elipses` se muestra la forma de detectar un marcador basado en 4 círculos.

## 10

En esta sesión vamos a extraer la matriz de cámara a partir del marcador utilizado en la sesión anterior, lo que nos permitirá añadir objetos virtuales tridimensionales a la escena y determinar la posición de la cámara en el espacio.

Nos vamos a la carpeta `code/pose`, donde encontraremos los siguientes ejemplos de código:

`pose0.py` incluye el código completo para extraer contornos, detectar el marcador poligonal, extraer la matriz de cámara y dibujar un cubo encima del marcador.

`pose1.py` hace lo mismo con funciones de umucv.

`pose2.py` trata de ocultar el marcador y dibuja un objeto que cambia de tamaño.

`pose3.py` explica la forma de proyectar una imagen en la escena escapando del plano del marcador.

`pose3D.py` es un ejemplo un poco más avanzado que utiliza el paquete pyqtgraph para mostrar en 3D la posición de la cámara en el espacio.

En el ejercicio **RA** puedes intentar que el comportamiento del objeto virtual dependa de acciones del usuario (p. ej. señalando con el ratón un punto del plano) o de objetos que se encuentran en la escena.

## 11

Breve introducción a scikit-learn y keras.

En primer lugar repasaremos algunos conceptos básicos en el notebook [machine learning](machine-learning.ipynb).

Esta sesión está dedicada a poner en marcha una red convolucional sencilla. La tarea que vamos a resolver es el reconocimiento de dígitos manuscritos. Por eso, en primer lugar es conveniente escribir unos cuantos números en una hoja de papel, con un bolígrafo que tenga un trazo no demasiado fino, y sin preocuparnos mucho de que estén bien escritos. Pueden tener distintos tamaños, pero no deben estar muy girados. Para desarrollar el programa y hacer pruebas cómodamente se puede trabajar con una imagen fija, pero la idea es que nuestro programa funcione con la cámara en vivo.

Trabajaremos en la carpeta [code/DL/CNN](../code/DL/CNN), donde tenemos las diferentes etapas de ejercicio y una imagen de prueba.

El primer paso es `digitslive-1.py` que simplemente encuentra las manchas de tinta que pueden ser posibles números.

En `digitslive-2.py` normalizamos el tamaño de las detecciones para poder utilizar la base de datos MNIST.

En `digitslive-3.py` implementamos un clasificador gaussiano con reducción de dimensión mediante PCA y lo ponemos en marcha con la imagen en vivo. (Funciona bastante bien pero, p.ej., en la imagen de prueba comete un error).

Finalmente, en `digitslive-4.py` implementamos la clasificación mediante una red convolucional mediante el paquete **keras**. Usamos unos pesos precalculados. (Esta máquina ya no comete el error anterior.)

Como siempre, en cada fase del ejercicio los comentarios explican el código que se va añadiendo.

Una vez conseguido esto, la sesión práctica tiene una segunda actividad que consiste en **entrenar los pesos** de (por ejemplo) esta misma red convolucional. Para hacerlo en nuestro ordenador sin perder la paciencia necesitamos una GPU con CUDA y libCUDNN. La instalación de todo lo necesario puede no ser trivial. 

Una alternativa muy práctica es usar [google colab](https://colab.research.google.com/), que proporciona gratuitamente máquinas virtuales con GPU y un entorno de notebooks jupyter (un poco modificados pero compatibles). Para probarlo, entrad con vuestra cuenta de google y abrid un nuevo notebook. En la opción de menú **Runtime** hay que seleccionar **Change runtime type** y en hardware accelerator ponéis GPU. En una celda del notebook copiáis directamente el contenido del archivo `cnntest.py` que hay en este mismo directorio donde estamos trabajando hoy. Al evaluar la celda se descargará la base de datos y se lanzará un proceso de entrenamiento. Cada epoch tarda unos 4s. Podéis comparar con lo que se consigue con la CPU en vuestro propio ordenador. Se puede lanzar un entrenamiento más completo, guardar los pesos y descargarlos a vuestra máquina.

Como curiosidad, podéis comparar con lo que conseguiría el OCR tesseract, y guardar algunos casos de dígitos que estén bien dibujados pero que la red clasifique mal.

Finalmente, entrenamos un autoencoder y comparamos el resultado con la reducción de dimensión PCA explicada al principio.

## 12

En esta sesión vamos a poner en marcha algunos modelos más avanzados de deep learning.

Los ejemplos de código se han probado sobre LINUX. En Windows o Mac puede ser necesario hacer modificaciones; para no perder mucho tiempo mi recomendación es probarlo primero en una máquina virtual.

Si tenéis una GPU nvidia reciente lo ideal es instalar CUDA y libCUDNN para conseguir una mayor velocidad de proceso. Si no tenéis GPU no hay ningún problema, todos los modelos funcionan con CPU. (Los ejercicios de deep learning que requieren entrenamiento son opcionales y se pueden entrenar en COLAB.)

Para ejecutar las máquinas inception, YOLO y el reconocimiento de caras necesitamos los siguientes paquetes:

    pip install  face_recognition  tensorflow==1.15.0  keras  easydict

La detección de marcadores corporales *openpose* requiere unos pasos de instalación adicionales que explicaremos más adelante.

(La versión 1.15.0 de tensorflow es necesaria para YOLO y openpose. Producirá algunos warnings sin mucha importancia. Si tenemos una versión más reciente de tensorflow podemos hacer `pip install --upgrade tensorflow=1.15.0` o crear un entorno de conda especial para este tema).

1) Para probar el **reconocimiento de caras** nos vamos a la carpeta code/DL/facerec. Debe estar correctamente instalado DLIB. 

En el directorio `gente` se guardan los modelos. Como ejemplo tenemos a los miembros de Monty Python:

    ./facerec.py --dev=dir:../../../images/monty-python*
    
(Recuerda que las imágenes seleccionadas con --dev=dir: se avanzan pinchando con el ratón en la ventana pequeña de muestra).

Puedes meter fotos tuyas y de tu familia en la carpeta `gente` para probar con la webcam o con otras fotos.

Con pequeñas modificaciones de este programa se puede resolver el ejercicio ANON: selecciona una cara en la imagen en vivo pinchando con el ratón para ocultarla  (emborronándola o pixelizándola) cuando se reconozca en las imágenes siguientes.

Esta versión del reconocimiento de caras no tiene aceleración con GPU (tal vez se puede configurar). Si reducimos un poco el tamaño de la imagen funciona con bastante fluidez.

2) Para probar la máquina **inception** nos movemos a la carpeta code/DL/inception.

    ./inception0.py
    
(Se descargará el modelo del la red). Se puede probar con las fotos incluidas en la carpeta con `--dev=dir:*.png`. La versión `inception1.py` captura en hilo aparte y muestra en consola las 5 categorías más probables.

Aunque se supone que consigue buenos resultados en las competiciones, sobre imágenes naturales comete bastante errores.

3) El funcionamiento de **YOLO** es mucho mejor. Nos vamos a la carpeta code/DL y ejecutamos lo siguiente para para descargar el código y los datos de esta máquina (y de openpose).

    bash get.sh
    
Nos metemos en code/DL/yolo y ejecutamos:

    /.yolo-v3.py
    
Se puede probar también con las imágenes de prueba incluidas añadiendo `--dev=dir:*.png`.

El artículo de [YOLO V3](https://pjreddie.com/media/files/papers/YOLOv3.pdf) es interesante. En la sección 5 el autor explica que abandonó esta línea de investigación por razones éticas. Os recomiendo que la leáis. Como curiosidad, hace unos días apareció [YOLO V4](https://arxiv.org/abs/2004.10934).

4) Para probar **openpose** nos vamos a code/DL/openpose. Los archivos necesarios ya se han descargado en el paso anterior, pero necesitamos instalar algunos paquetes. El proceso se explica en el README. 

En la carpeta `docker` hay un script para ejecutar una imagen docker que tiene instalados todos los paquetes que hemos estamos usando en la asignatura. Es experimental. No perdaís ahora tiempo con esto si no estáis familiarizados con docker.

El tema de deep learning en visión artificial es amplísimo. Para estudiarlo en detalle hace falta (como mínimo) una asignatura avanzada (master). Nuestro objetivo es familizarizarnos un poco con algunas de las máquinas preentrenadas disponibles para hacernos una idea de sus ventajas y limitaciones.

Si estáis interesados en estos temas el paso siguiente es adaptar alguno de estos modelos a un problema propio mediante "transfer learning", que consiste en utilizar las primeras etapas de una red preentrenada para transformar nuestros datos y ajustar un clasificador sencillo. Alternativamente, se puede reajustar los pesos de un modelo preentrenado, fijando las capas iniciales al principio. Para remediar la posible falta de ejemplos se utilizan técnicas de "data augmentation", que generan variantes de los ejemplos de entrenamiento con múltiples transformaciones.

mediapipe

UNET

## entrenar dlib

- (opcional) DLIB herramienta de etiquetado imglab. Entrenamiento de detector HOG SVM con herramientas de DLIB:

    - descargar y descomprimir dlib source
    - ir a los ejemplos/faces
    - meter dentro imglab (que hay que compilar pero tenemos versión precompilada en robot/material/va)
    - mostrar los training.xml y testing.xml (se pueden crear otros)
    - meter dentro train_detector.py y run_detector.py de code/hog
    - ./train_detector training.xml testing.xml  (crea detector.svm)
    - ./run_detector detector.svm --dev=dir:\*.jpg   (o también --dev=dir:/path/to/umucv/images/monty\*)
    

## correlation filter

Comentamos el método de detección de objetos por correlación cruzada, que es el mismo criterio que se usa para buscar la posición de *corners* en imágenes sucesivas, y luego vemos la demostración del discriminative correlation filter.

- crosscorr.py

- dcf.py

## flask server

El ejemplo `server.py` explica cómo hacer un servidor web sencillo con *flask* para enviar un pantallazo de la imagen actual de la webcam, y `mjpegserver.py` explica cómo hacer un servidor de streaming en formato mjpeg.

## telegram bot

Vamos a jugar con un bot de telegram que nos permite comunicarnos cómodamente con nuestro ordenador desde el teléfono móvil, sin necesidad de tener una dirección pública de internet.

Simplemente necesitamos:

    pip install python-telegram-bot

El ejemplo `bot/bot0.py` nos envía al teléfono la IP del ordenador (es útil si necesitamos conectarnos por ssh con una máquina que tiene IP dinámica).

El ejemplo `bot/bot1.py` explica la forma de enviar una imagen nuestro teléfono cuando ocurre algo. En este caso se envía cuando se pulsa una tecla, pero lo normal es detectar automáticamente algún evento con las técnicas de visión artificial que estamos estudiando.

El ejemplo `bot/bot2.py` explica la forma de hacer que el bot responda a comandos. El comando /hello nos devuelve el saludo, el comando /stop detiene el programa y el comando /image nos devuelve una captura de nuestra webcam. (Se ha usado la captura en un hilo). 

El ejemplo `bot/bot3.py` explica la forma de capturar comandos con argumentos y el procesamiento de una imagen enviada por el usuario.

Esta práctica es muy útil para enviar cómodamente a nuestros programas de visión artificial una imagen tomada con la cámara sin necesidad de escribir una aplicación específica para el móvil. Algunos ejercicios que estamos haciendo se pueden adaptar fácilmente para probarlos a través de un bot de este tipo.

Para crearos vuestro propio bot tenéis que contactar con el bot de telegram "BotFather", que os guiará paso a paso y os dará el token de acceso. Y luego el "IDBot" os dirá el id numérico de vuestro usuario.